# Linear Programming
##### by Jorge Almonacid


#### Introduction
In this notebook we'll resolve a linear programming problem with the help of the ortools library and provide a code that accepts matrix form problem inputs for a faster use.

## Logistics example
Firstly, let's get used to the library to then generalize it. In this example we'll use the pywraplp library from ortools in order to help us resolve the problem.

#### Problem formulation:
A company must ship products from two warehouses to three retail stores. The goal is to minimize the transportation cost while fulfilling store demand and not exceeding warhouse supply. We need to minimize the following problem then:

#### Data

$\begin{array}{l|llll}
\text{From \ To} & \text{Store 1} & \text{Store 2} & \text{Store 3} & \text{Supply}\\
\hline
\text{Warehouse} 1 & €4  & €6 & €9 & 100\\
\text{Warehouse} 2 & €5 & €4 & €7 & 120\\
\text{Demand} & 80 & 70 & 50 & 
\end{array}$


#### Constraints
##### Supply constraints:
$\begin{array}{c l}
x_{11} + x_{12} + x_{13} \leq 100&   (\text{Warehouse 1})\\
x_{21} + x_{22} + x_{23} \leq 120&   (\text{Warehouse 2})
\end{array}
$

##### Demand constraints:
$\begin{array}{c l}
x_{11} + x_{21}  = 80&   (\text{Store 1})\\
x_{12} + x_{22}  = 70&   (\text{Store 2}) \\
x_{13} + x_{23}  = 50&   (\text{Store 3})
\end{array}
$

##### Non-negativity:
$x_{ij} \ge 0\hspace{5mm} \forall\hspace{1mm} i,j$

##### Objective Function:
Minimize: $4x_{11} + 6x_{12} + 9x_{13} + 5x_{21} + 4x_{22} + 7x_{23}$


With $x_{ij}$ being the units shipped from warehouse i to store j.

## Code implementation
Firstly we import the libraries that we are going to use: pywraplp from ORTools

In [49]:
from ortools.linear_solver import pywraplp

Now initialize the solver with the class Solver.

In [50]:
solver = pywraplp.Solver("Logistics: ", pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

Once the solver class is created we can add the different variables and constraints associated to them. 


Firstly, to initialize the variables we use the IntVar method from the class solver, assign them a range, in our case $[0,\infty)$, and a name, x_11, x_22, .... Afterwards, we set the restrictions with the Constraint method, there are several ways to use it, some shown in the code below, but the main point is: Setting the interval de constraint ranges, and set the coefficients associated to each of the variables previously initialized.

In [51]:
# x_11 + x_12 + x_13 <= 100
# x_21 + x_22 + x_23 <= 120
# x_11 + x_21 = 80
# x_12 + x_22 = 70
# x-13 + x_23 = 50
# x_ij >= 0
# min: 4x_11+6x_12+9x_13+5x_21+4x_22+7x_23

# Variable initialization

x_11 = solver.IntVar(0,solver.infinity(), "x_11")
x_12 = solver.IntVar(0,solver.infinity(), "x_12")
x_13 = solver.IntVar(0,solver.infinity(), "x_13")

x_21 = solver.IntVar(0,solver.infinity(), "x_21")
x_22 = solver.IntVar(0,solver.infinity(), "x_22")
x_23 = solver.IntVar(0,solver.infinity(), "x_23")

# Restriction definitions

# Inequality constraints
constraint1 = solver.Constraint(0,100)
constraint1.SetCoefficient(x_11,1)
constraint1.SetCoefficient(x_12,1)
constraint1.SetCoefficient(x_13,1)

constraint2 = solver.Constraint(0,120)
constraint2.SetCoefficient(x_21,1)
constraint2.SetCoefficient(x_22,1)
constraint2.SetCoefficient(x_23,1)

# Equality constraints
solver.Add(sum([x_11, x_21])== 80)
# solver.Add(x_12 + x_22== 70)
solver.Add(x_13 + x_23== 50)

# Same as the one commented above but with the first method
constraint3 = solver.Constraint(70,70)
constraint3.SetCoefficient(x_12, 1)
constraint3.SetCoefficient(x_22, 1)

Now we define the objective function to minimize, and we just need to set the coefficients associated to each variable like before, thanks to the method Objective.

In [52]:
# Minimization function coefficient set

objective = solver.Objective()
objective.SetCoefficient(x_11, 4)
objective.SetCoefficient(x_12, 6)
objective.SetCoefficient(x_13, 9)
objective.SetCoefficient(x_21, 5)
objective.SetCoefficient(x_22, 4)
objective.SetCoefficient(x_23, 7)
objective.SetMinimization()

Finally, to get the problem solution we call the Solve method and print each of the values associated to each variable and the solution for the minimization problem.

In [53]:
# solver and solution print
result = solver.Solve()
print("Cost:", solver.Objective().Value())
print("x_12:", x_11.solution_value())
print("x_13:", x_12.solution_value())
print("x_11:", x_13.solution_value())
print("x_21:", x_21.solution_value())
print("x_22:", x_22.solution_value())
print("x_23:", x_23.solution_value())

Cost: 950.0
x_12: 80.0
x_13: 0.0
x_11: 0.0
x_21: 0.0
x_22: 70.0
x_23: 50.0


There's an alternative way to solve the problem, with the Minimize/Maximize method, where you just need to introduce the equation with the variables.

In [54]:
# solver.Minimize(4*x_11+6*x_12+9*x_13+5*x_21+4*x_22+7*x_23)

### Generalization
Now that we have seen how ORTools works for linear programming optimization problems, we can do a function that, given a matrix form problem, it resolves it with the methods used earlier. 

General formula that we try to generalize in the equation:
- $\textbf{Optimize}$
$c^Tx$
- Subject to:

$\hspace{20mm} \begin{cases} 
Ax \leq b \\ 
A_{eq}x = b_{eq} \\ 
x \geq 0 
\end{cases}$

There is one main function, optimize, and an auxiliary one, get_coeffs, the latter helps us with extracting the coefficients from a list and returns them as the coefficients associated to each variable, while the former just does the process we've done before automatically, with the help of the library and dictionaries.

In [55]:
# comparar con distinto número de almacenes, 1, 2, combinaciones de los 2, suponer que hay 1 tercer almacén
# Automatizar con una funcion que reciba las matrices de datos y devuelva el resultado


def get_coeffs(c):
    vars = {}
    s= 0
    for i in range(0,len(c)):
        vars["x{0}".format(s)] = c[i]
        s += 1
    return vars


def optimize(c, A_ub, b_ub, A_eq=None, b_eq=None, minmax = 0):
    """
    Optimizes a set problem given the constraints and coefficients from the optimization function in matrix form.
    Assuming discrete variables and every variable is equal or greater than 0.

    Args:
        c (list): Coefficients of the function to minimize
        A_ub (list): Coefficients for the lhs of the inequality constraints
        b_ub (list): Coefficients for the rhs of the inequality constraints
        A_eq (list): Coefficients for the lhs of the equality constraints
        b_eq (list): Coefficients for the rhs of the equality constraints
        minmax (int): Determines whether the problem is minimization or maximization (0 min, 1 max)

    Returns:
        sol (float): Solution to the optimization problem
        solution_dictionary (dict): Dictionary with the solution values associated to each variable used
    """

    # Initialization of the model and the variables
    solver = pywraplp.Solver("Optimization problem", pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    
    # Setting the problem as a minimization in case the one introduced is maximization problem by changing the sign
    if minmax == 1:
        c = [-elem for elem in c]
    
    # We get the coefficients from c and save the keys in a list
    vars = get_coeffs(c)
    intvar = list(vars.keys())
    intvars = {}
    
    # Initialize all the needed variables from 0 to ∞ and save them in a dictionary
    for i in range(len(vars)):
        intvars["x{0}".format(i)] = solver.IntVar(0,solver.infinity(), "{0}".format(intvar[i]))
  
    # Upper bound constraints
    constraints_ub = {}
    for i in range(len(A_ub)):
        constraints_ub["constraint{0}".format(i)] = solver.Constraint(0,b_ub[i])
        setconstraint = constraints_ub[list(constraints_ub.keys())[i]]
        for j in range(len(A_ub[i])):
            if A_ub != 0:
                setconstraint.SetCoefficient(intvars[list(intvars.keys())[j]],A_ub[i][j])
    
    # Equality constraints if we have
    if A_eq is not None:
        constraints_eq = {}
        for i in range(len(A_eq)):
            constraints_eq["constraint{0}".format(i)] = solver.Constraint(b_eq[i],b_eq[i])
            setconstraint = constraints_eq[list(constraints_eq.keys())[i]]
            for j in range(len(A_eq[i])):
                if A_eq != 0:
                    setconstraint.SetCoefficient(intvars[list(intvars.keys())[j]],A_eq[i][j])
    
    # Set objective function coefficients
    objective = solver.Objective()
    for i in range(len(vars)):
        objective.SetCoefficient(intvars[list(intvars.keys())[i]], c[i])
    
    # Show results
    print("Optimization completed")

    solver.Solve()
    sol = solver.Objective().Value()
    if minmax == 1:
        sol = -sol
        
    print(f"Function optimization value: {sol}")
    for i in range(len(vars)):
        print(f"{list(intvars.keys())[i]} = {round(intvars[list(intvars.keys())[i]].solution_value())}")
    
    solution_dictionary = {}
    for i in range(len(c)):
        solution_dictionary = {**solution_dictionary, f"{list(intvars.keys())[i]}": round(intvars[list(intvars.keys())[i]].solution_value())}

    return sol, solution_dictionary

Now we test if our program works, with the following set up:

In [56]:

c = [300,400]
A_ub = [[1,1],[120,200], [10,20]]
# A_eq = [[1,1]]
b_ub = [100,20000,1200]
# b_eq = [2000]

Optimized_cost, products = optimize(c,A_ub,b_ub,minmax=1) 

Optimization completed
Function optimization value: 32000.0
x0 = 80
x1 = 20
